# Ch5

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

sns.set()
%matplotlib inline

/Users/hiroaki/.pyenv/versions/3.6.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/hiroaki/.pyenv/versions/3.6.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## カテゴライズ変数のエンコーディング
- one-hot:全ての特徴量の中で、一つだけ1となり、ほかは0
- dummy-coding: one-hotの次元から不要な一次元を取り除く
- effect-coding: TODO

In [4]:
# onehotとdummy-encodeで都市空間から家賃を予測
from sklearn  import linear_model

df_rent=pd.DataFrame(
    {'City': ['SF', 'SF', 'SF', 'NYC', 'NYC', 'NYC', 
        'Seattle', 'Seattle', 'Seattle'],
        'Rent': [3999, 4000, 4001, 3499, 3500, 3501, 2499, 2500, 2501]})

df_rent

,City,Rent
0,SF,3999
1,SF,4000
2,SF,4001
3,NYC,3499
4,NYC,3500
5,NYC,3501
6,Seattle,2499
7,Seattle,2500
8,Seattle,2501


In [8]:
one_hot_city=pd.get_dummies(df_rent,prefix='city')
one_hot_city

,Rent,city_NYC,city_SF,city_Seattle
0,3999,0,1,0
1,4000,0,1,0
2,4001,0,1,0
3,3499,1,0,0
4,3500,1,0,0
5,3501,1,0,0
6,2499,0,0,1
7,2500,0,0,1
8,2501,0,0,1


In [56]:
dummy_city=pd.get_dummies(df_rent,prefix='city',drop_first=True)

dummy_city

,Rent,city_SF,city_Seattle
0,3999,1,0
1,4000,1,0
2,4001,1,0
3,3499,0,0
4,3500,0,0
5,3501,0,0
6,2499,0,1
7,2500,0,1
8,2501,0,1


In [55]:
# 線形モデルで禁じ
model=linear_model.LinearRegression()
model.fit(one_hot_city[['city_NYC','city_SF','city_Seattle']]
                   ,one_hot_city['Rent'])


print(model.coef_+model.intercept_)
print(df_rent.groupby('City').mean())

[3500. 4000. 2500.]
         Rent
City         
NYC      3500
SF       4000
Seattle  2500


- one-hotの場合、それぞれの係数にバイアスを足したものが平均となる

In [61]:
model.fit(dummy_city[['city_SF','city_Seattle']]
                  , dummy_city['Rent'])


print(model.coef_+model.intercept_)
print(df_rent.groupby('City').mean())

[4000. 2500.]
         Rent
City         
NYC      3500
SF       4000
Seattle  2500


- dummy_codingの場合、biasはdropした特徴量の平均となる

## カテゴリ数が非常に大きな場合、どのようにすればよいか
- カテゴリ数が多いと、スパースな行列になる。
- これをいかに密な行列に変換するか

### Feature Hashing

In [66]:
# feature hashingでどれぐらい容量が軽減されるのかを確認する

# yelpのユーザーデータの最初の5000行を読み込む
import json

jsons=[]
with open('./dataset/yelp_review.json') as file:
     for i in range(5000):
            jsons.append(json.loads(file.readline()))
            
yelp_revew_df=pd.DataFrame(jsons)

yelp_revew_df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw
1,AEx2SYEUJmTxVVB18LlCwA,0,2016-05-28,0,vkVSCC7xljjrAI4UGfnKEQ,5,Super simple place but amazing nonetheless. It...,0,bv2nCi5Qv5vroFiqKGopiw
2,VR6GpWIda3SfvPC-lg9H3w,0,2016-05-28,0,n6QzIUObkYshz4dz2QRJTw,5,Small unassuming place that changes their menu...,0,bv2nCi5Qv5vroFiqKGopiw
3,CKC0-MOWMqoeWf6s-szl8g,0,2016-05-28,0,MV3CcKScW05u5LVfF6ok0g,5,Lester's is located in a beautiful neighborhoo...,0,bv2nCi5Qv5vroFiqKGopiw
4,ACFtxLv8pGrrxMm6EgjreA,0,2016-05-28,0,IXvOzsEMYtiJI0CARmj77Q,4,Love coming here. Yes the place always needs t...,0,bv2nCi5Qv5vroFiqKGopiw


In [99]:
# business_idの数
m=len(yelp_revew_df.business_id.unique())

from sklearn.feature_extraction import FeatureHasher

h=FeatureHasher(n_features=m,input_type='string')
f=h.transform(yelp_revew_df.business_id)

print(m)
print(np.shape(f.toarray()))
f_arr=f.toarray()


from sys import getsizeof

print('original data size is',getsizeof(yelp_revew_df.business_id))
print('hashed data size is',getsizeof(f))

4477
(5000, 4477)
original data size is 395104
hashed data size is 56


### Bin Counting

In [65]:
# ctr予測のデータをbin countingで圧縮する
ctr_df=pd.read_csv('./dataset/ctr_test.csv')

ctr_df.head()

,id,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000017e+19,14103100,1005,0,235ba823,f6ebf28e,f028772b,ecad2386,7801e8d9,07d7df22,...,1,0,8330,320,50,761,3,175,100075,23
1,1.000018e+19,14103100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,1,0,22676,320,50,2616,0,35,100083,51
2,1.000055e+19,14103100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,1,0,22676,320,50,2616,0,35,100083,51
3,1.000109e+19,14103100,1005,0,85f751fd,c4e18dd6,50e219e0,51cedd4e,aefc06bd,0f2161f8,...,1,0,18648,320,50,1092,3,809,100156,61
4,1.000138e+19,14103100,1005,0,85f751fd,c4e18dd6,50e219e0,9c13b419,2347f47a,f95efa07,...,1,0,23160,320,50,2667,0,47,-1,221
